https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/export/?flg=fr&sort=date&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQVZHIiwieUF4aXMiOiJ0YyIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6IiNGRjUxNUEifV0sInhBeGlzIjoiZGF0ZSIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6ImRheSIsInNvcnQiOiIiLCJjb25maWciOnsiZGF0YXNldCI6ImRvbm5lZXMtc3lub3AtZXNzZW50aWVsbGVzLW9tbSIsIm9wdGlvbnMiOnsiZmxnIjoiZnIifX19XSwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D&q.timerange.date=date:%5B2019-08-31T22:00:00Z+TO+2022-10-01T21:59:59Z%5D&location=2,-15.59255,38.57442&basemap=jawg.light

In [7]:
import pandas as pd
import matplotlib.pyplot as plt


In [11]:
# import wget

# site_url = 'https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/download/?format=csv&q=date:%5B2019-08-31T22:00:00Z+TO+2022-10-01T21:59:59Z%5D&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B'
# file_name = wget.download(site_url)



In [ ]:
# !mv donnees-synop-essentielles-omm.csv data_meteo.csv

In [13]:
df = pd.read_csv('data_meteo.csv',sep=';')
df.head()

,ID OMM station,Date,Pression au niveau mer,Variation de pression en 3 heures,Type de tendance barométrique,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Point de rosée,Humidité,...,Altitude,communes (name),communes (code),EPCI (name),EPCI (code),department (name),department (code),region (name),region (code),mois_de_l_annee
0,61980,2022-03-09T22:00:00+01:00,101140.0,-110.0,7.0,170.0,1.7,298.25,295.15,83.0,...,8,Sainte-Marie,97418,CA Intercommunale du Nord de la Réunion (CINOR),249740119.0,La Réunion,974,La Réunion,4.0,3
1,7577,2022-03-10T01:00:00+01:00,102530.0,40.0,0.0,0.0,0.0,277.15,275.45,89.0,...,73,Montélimar,26198,CA Montélimar Agglomération,200040459.0,Drôme,26,Auvergne-Rhône-Alpes,84.0,3
2,81415,2022-03-10T07:00:00+01:00,101200.0,130.0,7.0,0.0,0.0,295.45,295.45,100.0,...,106,Maripasoula,97353,CC de l'Ouest Guyanais,249730037.0,Guyane,973,Guyane,3.0,3
3,7240,2022-03-10T10:00:00+01:00,101970.0,40.0,1.0,130.0,4.6,283.45,278.65,72.0,...,108,Parçay-Meslay,37179,Tours Métropole Val de Loire,243700754.0,Indre-et-Loire,37,Centre-Val de Loire,24.0,3
4,7299,2022-03-10T10:00:00+01:00,102530.0,-20.0,8.0,210.0,0.6,280.85,271.35,51.0,...,263,Blotzheim,68042,CA Saint-Louis Agglomération,200066058.0,Haut-Rhin,68,Grand Est,44.0,3
